In [1]:
import numpy as np
import configparser
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, core, Dropout, concatenate, BatchNormalization,Activation
from keras.callbacks import ModelCheckpoint,TensorBoard

from helpers import load_hdf5

Using TensorFlow backend.


In [2]:
config = configparser.RawConfigParser()
config.read('configuration.txt')

#train data
train_imgs = config.get('data paths', 'train_imgs')
train_gt = config.get('data paths', 'train_gt')

#training settings
N_epochs = int(config.get('training settings', 'N_epochs'))
batch_size = int(config.get('training settings', 'batch_size'))

#patch properties
patch_height = int(config.get('data attributes','patch_height'))
patch_width = int(config.get('data attributes','patch_width'))
patch_num = int(config.get('data attributes','patch_num'))

In [3]:
def masks_reshape(masks):
    
    im_h = masks.shape[1]
    im_w = masks.shape[2]
    
    masks = np.reshape(masks,(masks.shape[0],im_h*im_w))
    new_masks = np.empty((masks.shape[0],im_h*im_w,2), dtype = 'float16')
    for i in range(masks.shape[0]):
        for j in range(im_h*im_w):
            if  masks[i,j] == 1:
                new_masks[i,j,0]=1
                new_masks[i,j,1]=0
            
            else:
                new_masks[i,j,0]=0
                new_masks[i,j,1]=1
                
    return new_masks 

In [4]:
def getShallowUnet(patch_height,patch_width,n_ch):
    #
    inputs = Input((patch_height, patch_width,n_ch))
    #
    conv1 = Conv2D(32, (3, 3), padding="same", activation="relu")(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), padding="same", activation="relu")(conv1)
    print(conv1.shape)
    #
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    print(pool1.shape)
    #
    conv2 = Conv2D(64, (3, 3), padding="same", activation="relu")(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), padding="same", activation="relu")(conv2)
    print(conv2.shape)
    #
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    print(pool2.shape)
    #
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    print(conv3.shape)
    #
    up1 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv2], axis=-1)
    print(up1.shape)
    #
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    print(conv4.shape)
    #
    up2 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv1], axis=-1)
    print(up2.shape)
    #
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    print(conv5.shape)
    #
    conv6 = Conv2D(2, (1, 1), activation='relu', padding='same')(conv5)
    print(conv6.shape)
    conv6 = core.Reshape((2,patch_height*patch_width))(conv6)
    print(conv6.shape)
    conv6 = core.Permute((2,1))(conv6)
    print(conv6.shape)
    #
    conv7 = core.Activation('softmax')(conv6)
    print(conv7.shape)

    model = Model(inputs=inputs, outputs=conv7)

    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()

    return model

In [4]:
#Unet with batch normalization
def getShallowUnet2(patch_height,patch_width,n_ch):
    #
    inputs = Input((patch_height, patch_width,n_ch))
    #
    conv1 = Conv2D(32, (3, 3), padding="same")(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Dropout(0.2)(conv1)
    
    conv1 = Conv2D(32, (3, 3), padding="same")(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    print(conv1.shape)
    #
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    print(pool1.shape)
    #
    conv2 = Conv2D(64, (3, 3), padding="same")(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Dropout(0.2)(conv2)
    
    conv2 = Conv2D(64, (3, 3), padding="same")(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    print(conv2.shape)
    #
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    print(pool2.shape)
    #
    conv3 = Conv2D(128, (3, 3), padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Dropout(0.2)(conv3)
    
    conv3 = Conv2D(128, (3, 3), padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    print(conv3.shape)
    #
    up1 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv2], axis=-1)
    print(up1.shape)
    #
    conv4 = Conv2D(64, (3, 3),padding='same')(up1)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Dropout(0.2)(conv4)
    
    conv4 = Conv2D(64, (3, 3), padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    print(conv4.shape)
    #
    up2 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv1], axis=-1)
    print(up2.shape)
    #
    conv5 = Conv2D(32, (3, 3), padding='same')(up2)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Dropout(0.2)(conv5)
    
    conv5 = Conv2D(32, (3, 3), padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    print(conv5.shape)
    #
    conv6 = Conv2D(2, (1, 1), activation='relu', padding='same')(conv5)
    print(conv6.shape)
    conv6 = core.Reshape((2,patch_height*patch_width))(conv6)
    print(conv6.shape)
    conv6 = core.Permute((2,1))(conv6)
    print(conv6.shape)
    #
    conv7 = core.Activation('softmax')(conv6)
    print(conv7.shape)

    model = Model(inputs=inputs, outputs=conv7)

    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()

    return model

In [5]:
model = getShallowUnet2(patch_width, patch_height, 1)

W0819 13:23:29.342400 47709276951424 deprecation_wrapper.py:118] From /software/tensorflow/2.0.0b/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 13:23:29.367875 47709276951424 deprecation_wrapper.py:118] From /software/tensorflow/2.0.0b/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 13:23:29.373198 47709276951424 deprecation_wrapper.py:118] From /software/tensorflow/2.0.0b/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 13:23:29.403563 47709276951424 deprecation_wrapper.py:118] From /software/tensorflow/2.0.0b/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v

(?, 32, 32, 32)
(?, 16, 16, 32)


W0819 13:23:32.954913 47709276951424 deprecation_wrapper.py:118] From /software/tensorflow/2.0.0b/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.



(?, 16, 16, 64)
(?, 8, 8, 64)
(?, 8, 8, 128)
(?, 16, 16, 192)
(?, 16, 16, 64)
(?, 32, 32, 96)


W0819 13:23:33.486143 47709276951424 deprecation_wrapper.py:118] From /software/tensorflow/2.0.0b/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



(?, 32, 32, 32)
(?, 32, 32, 2)
(?, 2, 1024)
(?, 1024, 2)
(?, 1024, 2)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
_______________________________________

In [6]:
X_train = load_hdf5(train_imgs)
X_train = X_train.reshape((X_train.shape[0], patch_height, patch_width, 1))

Y_train = load_hdf5(train_gt)
Y_train = Y_train.reshape((Y_train.shape[0], patch_height, patch_width,1))
Y_train = masks_reshape(Y_train)

In [7]:
X_train.shape

(38000, 32, 32, 1)

In [9]:
#train
model = getShallowUnet2(patch_width, patch_height,1)
json_string = model.to_json()
open('model.json', 'w').write(json_string)
checkpointer = ModelCheckpoint('bestWeights1.h5', verbose=1, monitor='val_loss', mode='auto', save_best_only=True) 
tbCallback = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

model.fit(X_train, Y_train, epochs = N_epochs, batch_size = batch_size, verbose=2, shuffle=True, validation_split=0.15, callbacks=[checkpointer,tbCallback])
model.save_weights('lastWeights1.h5', overwrite=True)

(?, 32, 32, 32)
(?, 16, 16, 32)
(?, 16, 16, 64)
(?, 8, 8, 64)
(?, 8, 8, 128)
(?, 16, 16, 192)
(?, 16, 16, 64)
(?, 32, 32, 96)
(?, 32, 32, 32)
(?, 32, 32, 2)
(?, 2, 1024)
(?, 1024, 2)
(?, 1024, 2)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 1)    0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 32, 32, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 32, 32, 32)   128         conv2d_23[0][0]                  
__________________________________________________________________________________________________
activation_2

Train on 32300 samples, validate on 5700 samples
Epoch 1/60
 - 58s - loss: 0.3598 - acc: 0.8397 - val_loss: 0.2702 - val_acc: 0.8661

Epoch 00001: val_loss improved from inf to 0.27019, saving model to bestWeights12.h5
Epoch 2/60
 - 56s - loss: 0.3000 - acc: 0.8549 - val_loss: 0.2622 - val_acc: 0.8755

Epoch 00002: val_loss improved from 0.27019 to 0.26220, saving model to bestWeights12.h5
Epoch 3/60
 - 57s - loss: 0.2867 - acc: 0.8608 - val_loss: 0.2436 - val_acc: 0.8805

Epoch 00003: val_loss improved from 0.26220 to 0.24357, saving model to bestWeights12.h5
Epoch 4/60
 - 57s - loss: 0.2799 - acc: 0.8643 - val_loss: 0.2446 - val_acc: 0.8819

Epoch 00004: val_loss did not improve from 0.24357
Epoch 5/60
 - 57s - loss: 0.2756 - acc: 0.8663 - val_loss: 0.2373 - val_acc: 0.8844

Epoch 00005: val_loss improved from 0.24357 to 0.23735, saving model to bestWeights12.h5
Epoch 6/60
 - 57s - loss: 0.2729 - acc: 0.8679 - val_loss: 0.2398 - val_acc: 0.8860

Epoch 00006: val_loss did not improve 

 - 58s - loss: 0.2381 - acc: 0.8863 - val_loss: 0.2375 - val_acc: 0.8862

Epoch 00058: val_loss did not improve from 0.23094
Epoch 59/60
 - 58s - loss: 0.2377 - acc: 0.8865 - val_loss: 0.2404 - val_acc: 0.8863

Epoch 00059: val_loss did not improve from 0.23094
Epoch 60/60
 - 58s - loss: 0.2376 - acc: 0.8865 - val_loss: 0.2367 - val_acc: 0.8868

Epoch 00060: val_loss did not improve from 0.23094
